# Browser Automation Homework ROUND 2
Due 7-16<br>
Completed by: **CAROL MORENO**

We're going to the United States trademark data base and going to collect all of Nike (or any company's) active trademarks.

We only want the live trademarks, and we want the `serial` number and the link to an image of each trademark. Feel free to collect the `wordmark` if you like.


Note: if you get asked if you're a bot, just complete the challenges manually.

In [5]:
import os
import random
import time

from playwright.async_api import async_playwright, expect, Keyboard

In [6]:
os.makedirs('data/', exist_ok=True)

In [7]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'

In [18]:
async def open_browser(headless=False, user_agent=user_agent):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Open firefox browser, can use chromium (chrome) or others
    browser = await playwright.chromium.launch(headless=headless)

    # set a user agent
    context = await browser.new_context(user_agent=user_agent)
  
    # Create a new browser window
    page = await browser.new_page()

    return browser, page

In [19]:
driver, page = await open_browser()

In [29]:
url = 'https://tmsearch.uspto.gov/search/'
await page.goto(url)

<Response url='https://tmsearch.uspto.gov/search/' request=<Request url='https://tmsearch.uspto.gov/search/' method='GET'>>

Only search by the "Owner" of each trademark

In [30]:
xpath_search_options = '//*[@id="mat-select-value-3"]'
filter = page.locator(xpath_search_options)

In [31]:
await filter.click()

In [32]:
xpath_option = '//span[contains(text(), "Owner")]'

In [33]:
await page.locator(xpath_option).click()

Find the search bar and input the search

In [34]:
xpath_searchbar = '//input[@aria-label="Search trademarks"]'

In [35]:
search = page.locator(xpath_searchbar)

In [36]:
# Change this if you like
company = 'Nike'

In [37]:
await search.fill(company)

Make the search

In [38]:
await page.keyboard.press("Enter")

Filter to only live contracts
<br>In other words, filter out the dead ones.

In [39]:
xpath_filter = '//*[@id="statusDead"]'

In [40]:
dead_button = page.locator(xpath_filter)

In [42]:
# click it
await dead_button.click()

# Save results

In [46]:
# how to save what the emulator sees
source = await page.content()
with open(f'data/trademarks_{company}.html', 'w') as f:
    f.write(source)

GO TO THE NEXT PAGE AND REPEAT

In [47]:
# this is the next page button
xpath_next = '//i[contains(text(), "navigate_next")]'

In [48]:
# What is the xpath for a result, make sure this is visible before going to next page.
xpath_result = '//*[@id="resultsRow"]'

In [49]:
next_button = page.locator(xpath_next)

In [50]:
await expect(next_button).to_be_visible()

In [51]:
await next_button.click()

In [ ]:
# Here we'll put it all together and iterate through.
# There are many ways to do this
i = 1
collect = True
while collect:
        try:
            # Wait briefly to see if next button is present
            await page.wait_for_selector(xpath_next, timeout=100)
            
            # check the next button is visible
            if await page.locator(xpath_next).is_visible():
                # click the next button
                next_button = page.locator(xpath_next)
                await next_button.click()
                # make sure the search result is visible
                await page.locator(xpath_result).is_visible()
                # save the contents
                source = await page.content()
                with open(f'data/trademarks_{company}_{i}.html', 'w') as f:
                    f.write(source)
                    i += 1
            else:
                collect = False
                break
        except TimeoutError:
            print("No more pages found — stopping loop.")
            collect = False
            break

## Parse the contents

In [57]:
import glob
import pandas as pd
from lxml import etree, html

In [58]:
# list the scraped pages to pars them
files = glob.glob('data/trademarks*.html')

In [70]:
# A trademark entry
xpath_trademark_cards = '//*[contains(@class, "card m-2 result-card ng-star-inserted")]'
# the serial number
xpath_serial = './/div[5]/div[2]/span'
xpath_worldmark = ".//span[@class='clickable']/text()"

In [71]:
data = []
for fn in files:
    # read into a XML tree from a string
    tree = html.fromstring(open(fn).read())
    trademarks = tree.xpath(xpath_trademark_cards)
    for trademark in trademarks:
        worldmark = trademark.xpath(xpath_worldmark)
        serial = trademark.xpath(xpath_serial)
        img = trademark.xpath('.//img') # get the images also

        worldmark = worldmark[0].strip() if worldmark_ else None
        serial = serial[0].text_content().strip() if serial_ else None
        
        if img:
            img_url = img[0].get('src',)
        row = { 'worldmark': worldmark,
                'serial' : serial,
               'img_url': img_url}
        data.append(row)

In [72]:
df = pd.DataFrame(data)
df.head(5)

,worldmark,serial,img_url
0,STRUCTURE,98797467,https://tmcms-docs.uspto.gov/cases/98797467/ma...
1,NIKE AIR VAPORMAX,86315323,https://tmcms-docs.uspto.gov/cases/86315323/ma...
2,SHIELD-TEK,87111608,https://tmcms-docs.uspto.gov/cases/87111608/ma...
3,NILUFAR,79376276,https://tmcms-docs.uspto.gov/cases/79376276/ma...
4,1NKNSC,98571679,https://tmcms-docs.uspto.gov/cases/98571679/ma...


In [73]:
df.to_csv('data/trademarks.csv', index=False)